#### To save the state of the file automatically every 60 seconds

In [1]:
%autosave 60

Autosaving every 60 seconds


#### Importing all the required libraries

In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

import statsmodels.api as sm
from statsmodels.tools.eval_measures import meanabs
import dask.dataframe as dd

#### To read data from the csv file and converting it to multiple parquet files in a directory. <font color='red'>Need to run only once.</font>

In [ ]:
# batteries = dd.read_csv("/home/porwal/AAP/batteries/pi/batteries_pi.csv", dtype={'ON_HAND': 'object',
#        'PLATFORM_CLUSTER_NAME': 'float64'})

# batteries.to_parquet("/home/porwal/AAP/batteries/pi/parquet/", engine="pyarrow")

#### Reading the file and filtering out stores with store number 1020

In [3]:
batteries = dd.read_parquet("/home/porwal/AAP/batteries/pi/parquet/", engine="pyarrow", dtype={'MPOG_ID': 'object'})

batteries= batteries[batteries['STORE_NUMBER'] != 1020]
batteries

,SKU_NUMBER,STORE_NUMBER,PART_TYPE,PLATFORM_CLUSTER_NAME,APPLICATION_COUNT,POP_EST_CY,POP_DENSITY_CY,PCT_WHITE,AGE,PCT_COLLEGE,PCT_BLUE_COLLAR,MEDIAN_HOUSEHOLD_INCOME,ESTABLISHMENTS,SKU_EXISTENCE_PY,SKU_EXISTENCE_CY,SKU_STORE_PDQ_PY,SKU_STORE_PDQ_CY,TOTAL_VIO_PY,TOTAL_VIO_CY,UNADJUSTED_TOTAL_VIO_PY,UNADJUSTED_TOTAL_VIO_CY,LOST_QTY_PY,LOST_QTY_CY,SS_SALES_PY,SS_SALES_CY,AVG_CLUSTER_UNIT_SALES_PY,AVG_CLUSTER_UNIT_SALES_CY,AVG_CLUSTER_LOST_SALES_PY,AVG_CLUSTER_LOST_SALES_CY,ROAD_QUALITY_INDEX,VIO_COMPARED_TO_CLUSTER_PY,VIO_COMPARED_TO_CLUSTER_CY,ADJUSTED_AVG_CLUSTER_SALES_PY,ADJUSTED_AVG_CLUSTER_SALES_CY,QTY_SOLD_PPY,QTY_SOLD_PY,QTY_SOLD_CY,AVG_CLUSTER_TOTAL_SALES_PY,AVG_CLUSTER_TOTAL_SALES_CY,SALES_SIGNAL_PY,SALES_SIGNAL_CY,FAILURE_SALES_PY,FAILURE_SALES_CY,LIFECYCLE_PY,LIFECYCLE_CY,ADJUSTED_LIFECYCLE_PY,ADJUSTED_LIFECYCLE_CY,PCT_OF_LIFECYCLE_REMAINING,LIFECYCLE_PRE_PEAK_POST,ADJ_AVG_CLUSTER_LOST_SALES_PY,ADJ_AVG_CLUSTER_LOST_SALES_CY,ADJ_AVG_CLUSTER_TOTAL_SALES_PY,ADJ_AVG_CLUSTER_TOTAL_SALES_CY,SOLD_SINCE_MAXI,cy_unit_sales,py_unit_sales,ppy_unit_sales,cy_gross_sales,py_gross_sales,ppy_gross_sales,cy_sales_cost,py_sales_cost,ppy_sales_cost,other_unit_sales,other_gross_sales,other_sales_cost,PROJECTED_GROWTH_PCT_PY,PROJECTED_GROWTH_PCT_CY,OTHER_UNIT_PLS_LOST_SALES_PPY,OTHER_UNIT_PLS_LOST_SALES_PY,OTHER_UNIT_PLS_LOST_SALES_CY,LOOKUP_CNT_PY,LOOKUP_CNT_CY,WEIGHTED_LOOKUP_CNT_PY,WEIGHTED_LOOKUP_CNT_CY,CY_PERIODS_IN_STOCK,WAIT_TIME_BUCKET,MPOG_ID,DATA_CLASSIFICATION,cy_qty_sold_transfer,cy_qty_sold_on_hand,py_qty_sold_transfer,py_qty_sold_on_hand,ppy_qty_sold_transfer,ppy_qty_sold_on_hand,ON_HAND,fy_ts_forecast,cy_ts_forecast
npartitions=37,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,object,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,object,float64,float64,float64,float64,float64,float64,object,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
batteries.head()

,SKU_NUMBER,STORE_NUMBER,PART_TYPE,PLATFORM_CLUSTER_NAME,APPLICATION_COUNT,POP_EST_CY,POP_DENSITY_CY,PCT_WHITE,AGE,PCT_COLLEGE,...,DATA_CLASSIFICATION,cy_qty_sold_transfer,cy_qty_sold_on_hand,py_qty_sold_transfer,py_qty_sold_on_hand,ppy_qty_sold_transfer,ppy_qty_sold_on_hand,ON_HAND,fy_ts_forecast,cy_ts_forecast
0,11151759,3450,BATTERY,23.0,0,11437.54,0.004853,0.03,35.32,0.14,...,NON PR NO EPC,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0
1,2040158,4237,None,18.0,0,26363.10,0.004392,0.17,33.61,0.13,...,NON PR NO EPC,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0
2,11151759,1334,BATTERY,24.0,0,10099.36,0.006776,0.01,34.12,0.12,...,NON PR NO EPC,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0
3,2040158,1231,None,0.0,0,14755.78,0.006646,0.05,35.03,0.10,...,NON PR NO EPC,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,0.0
4,11151759,3590,BATTERY,55.0,0,10380.87,0.004394,0.35,34.26,0.17,...,NON PR NO EPC,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0


#### Dropping all the columns containing data from the current year except QTY_SOLD_CY and PROJECTED_GROWTH_PCT_CY

In [5]:
drop_columns = []
for i in batteries.columns:
    if ('_cy' in i.lower() or 'cy_' in i.lower()) and i!='QTY_SOLD_CY' and i!='PROJECTED_GROWTH_PCT_CY':
        drop_columns.append(i)
    else:
        continue

In [6]:
len(drop_columns)

29

In [7]:
batteries.columns

Index(['SKU_NUMBER', 'STORE_NUMBER', 'PART_TYPE', 'PLATFORM_CLUSTER_NAME',
       'APPLICATION_COUNT', 'POP_EST_CY', 'POP_DENSITY_CY', 'PCT_WHITE', 'AGE',
       'PCT_COLLEGE', 'PCT_BLUE_COLLAR', 'MEDIAN_HOUSEHOLD_INCOME',
       'ESTABLISHMENTS', 'SKU_EXISTENCE_PY', 'SKU_EXISTENCE_CY',
       'SKU_STORE_PDQ_PY', 'SKU_STORE_PDQ_CY', 'TOTAL_VIO_PY', 'TOTAL_VIO_CY',
       'UNADJUSTED_TOTAL_VIO_PY', 'UNADJUSTED_TOTAL_VIO_CY', 'LOST_QTY_PY',
       'LOST_QTY_CY', 'SS_SALES_PY', 'SS_SALES_CY',
       'AVG_CLUSTER_UNIT_SALES_PY', 'AVG_CLUSTER_UNIT_SALES_CY',
       'AVG_CLUSTER_LOST_SALES_PY', 'AVG_CLUSTER_LOST_SALES_CY',
       'ROAD_QUALITY_INDEX', 'VIO_COMPARED_TO_CLUSTER_PY',
       'VIO_COMPARED_TO_CLUSTER_CY', 'ADJUSTED_AVG_CLUSTER_SALES_PY',
       'ADJUSTED_AVG_CLUSTER_SALES_CY', 'QTY_SOLD_PPY', 'QTY_SOLD_PY',
       'QTY_SOLD_CY', 'AVG_CLUSTER_TOTAL_SALES_PY',
       'AVG_CLUSTER_TOTAL_SALES_CY', 'SALES_SIGNAL_PY', 'SALES_SIGNAL_CY',
       'FAILURE_SALES_PY', 'FAILURE_SALES_CY', '

#### Dropping all the redundant, categorical and unnecessary columns

In [8]:
drop_columns.extend(['PLATFORM_CLUSTER_NAME','UNADJUSTED_TOTAL_VIO_PY', 'UNADJUSTED_TOTAL_VIO_CY', 
                     'SS_SALES_PY', 'FAILURE_SALES_PY','LIFECYCLE_PY', 'AVG_CLUSTER_LOST_SALES_PY', 
                     'other_unit_sales', 'other_gross_sales', 'AVG_CLUSTER_TOTAL_SALES_PY', 'other_sales_cost',
                     'LOOKUP_CNT_PY', 'WEIGHTED_LOOKUP_CNT_PY', 'py_qty_sold_transfer', 'py_qty_sold_on_hand',
                     'ppy_qty_sold_transfer', 'ppy_qty_sold_on_hand', 'ON_HAND', 'fy_ts_forecast','MPOG_ID', 
                     'DATA_CLASSIFICATION', 'PART_TYPE', 'WAIT_TIME_BUCKET','py_unit_sales','ppy_unit_sales',
                     'ppy_gross_sales', 'py_sales_cost','ppy_sales_cost','ROAD_QUALITY_INDEX','TOTAL_VIO_PY','ESTABLISHMENTS'])
batteries = batteries.drop(drop_columns,axis=1)
batteries.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 29 entries, SKU_NUMBER to OTHER_UNIT_PLS_LOST_SALES_PY
dtypes: float64(21), int64(8)

#### Imputation

In [9]:
batteries.OTHER_UNIT_PLS_LOST_SALES_PY = batteries.OTHER_UNIT_PLS_LOST_SALES_PY.fillna(0)
batteries.OTHER_UNIT_PLS_LOST_SALES_PPY = batteries.OTHER_UNIT_PLS_LOST_SALES_PPY.fillna(0)
batteries.PROJECTED_GROWTH_PCT_CY = batteries.PROJECTED_GROWTH_PCT_CY.fillna('mean')
batteries.PROJECTED_GROWTH_PCT_PY = batteries.PROJECTED_GROWTH_PCT_PY.fillna('mean')

#### Deleting all the SKUs with zero gross sales in the previous year and imputing negative values of QTY_SOLD_CY to zero

In [10]:
sku_list = set(batteries[batteries['py_gross_sales']!=0]['SKU_NUMBER'])

batteries = batteries[batteries['SKU_NUMBER'].isin(sku_list)]

batteries[batteries['QTY_SOLD_CY']<0]['QTY_SOLD_CY'] = 0

In [11]:
batteries.PROJECTED_GROWTH_PCT_PY = batteries.PROJECTED_GROWTH_PCT_PY.astype(float)
batteries.PROJECTED_GROWTH_PCT_CY = batteries.PROJECTED_GROWTH_PCT_CY.astype(float)

#### Defining predictor and target variables

In [12]:
y = batteries['QTY_SOLD_CY'].copy()
x = batteries.drop(['QTY_SOLD_CY','SKU_NUMBER','STORE_NUMBER'],axis=1).copy()

#### Converting the Dask dataframes back to Pandas dataframes

In [13]:
x = x.compute()
y = y.compute()
print(type(x))
print(type(y))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [14]:
num_cols = []
cat_cols = []
for i in x.columns:
    if x[i].dtype=='object':
        cat_cols.append(i)
    else:
        num_cols.append(i)
# num_cols

In [15]:
x.columns

Index(['APPLICATION_COUNT', 'PCT_WHITE', 'AGE', 'PCT_COLLEGE',
       'PCT_BLUE_COLLAR', 'MEDIAN_HOUSEHOLD_INCOME', 'SKU_EXISTENCE_PY',
       'SKU_STORE_PDQ_PY', 'LOST_QTY_PY', 'AVG_CLUSTER_UNIT_SALES_PY',
       'VIO_COMPARED_TO_CLUSTER_PY', 'ADJUSTED_AVG_CLUSTER_SALES_PY',
       'QTY_SOLD_PPY', 'QTY_SOLD_PY', 'SALES_SIGNAL_PY',
       'ADJUSTED_LIFECYCLE_PY', 'PCT_OF_LIFECYCLE_REMAINING',
       'LIFECYCLE_PRE_PEAK_POST', 'ADJ_AVG_CLUSTER_LOST_SALES_PY',
       'ADJ_AVG_CLUSTER_TOTAL_SALES_PY', 'SOLD_SINCE_MAXI', 'py_gross_sales',
       'PROJECTED_GROWTH_PCT_PY', 'PROJECTED_GROWTH_PCT_CY',
       'OTHER_UNIT_PLS_LOST_SALES_PPY', 'OTHER_UNIT_PLS_LOST_SALES_PY'],
      dtype='object')

#### Standardization

In [16]:
all_cols = x.columns

ct = ColumnTransformer([
        ('scaler', StandardScaler(), all_cols)
    ], remainder='passthrough')

x = pd.DataFrame(ct.fit_transform(x), columns=all_cols)

In [18]:
ex_cols=['QTY_SOLD_PY', 'QTY_SOLD_PPY']
t_cols = x.columns
for col1 in ex_cols:
    for col2 in t_cols:
        x[col1 + '&' + col2] = x[col1]*x[col2]

#### Splitting the data into random training and test subsets

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [21]:
y_train = y_train.tolist()
y_test = y_test.tolist()

#### Performing Ordinary Least Squares on the training data using statsmodels

In [23]:
%%time

#x = sm.add_constant(x)
olsmodel = sm.OLS(y_train,x_train)
results = olsmodel.fit()

results.summary()

CPU times: user 2min 27s, sys: 39.3 s, total: 3min 6s
Wall time: 4.98 s


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.912
Model:                            OLS   Adj. R-squared (uncentered):              0.912
Method:                 Least Squares   F-statistic:                          2.084e+05
Date:                Thu, 31 Mar 2022   Prob (F-statistic):                        0.00
Time:                        14:38:20   Log-Likelihood:                     -4.0737e+06
No. Observations:             1436348   AIC:                                  8.148e+06
Df Residuals:                 1436277   BIC:                                  8.148e+06
Df Model:                          71                                                  
Covariance Type:            nonrobust                                                  
===============================================================================================================
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
APPLICATION_COUNT                               0.0359      0.006      6.302      0.000       0.025       0.047
PCT_WHITE                                       0.0005      0.004      0.112      0.911      -0.008       0.009
AGE                                             0.0605      0.005     12.426      0.000       0.051       0.070
PCT_COLLEGE                                     0.0281      0.006      4.780      0.000       0.017       0.040
PCT_BLUE_COLLAR                                 0.0868      0.007     12.052      0.000       0.073       0.101
MEDIAN_HOUSEHOLD_INCOME                        -0.0023      0.004     -0.523      0.601      -0.011       0.006
SKU_EXISTENCE_PY                              -15.6590      0.046   -339.897      0.000     -15.749     -15.569
SKU_STORE_PDQ_PY                               17.3578      0.048    362.556      0.000      17.264      17.452
LOST_QTY_PY                                     0.0094      0.029      0.319      0.750      -0.048       0.067
AVG_CLUSTER_UNIT_SALES_PY                       1.0207      0.021     49.190      0.000       0.980       1.061
VIO_COMPARED_TO_CLUSTER_PY                      0.0589      0.004     14.936      0.000       0.051       0.067
ADJUSTED_AVG_CLUSTER_SALES_PY                1.202e+08   1.26e+10      0.010      0.992   -2.46e+10    2.48e+10
QTY_SOLD_PPY                                  -12.0829      0.059   -204.370      0.000     -12.199     -11.967
QTY_SOLD_PY                                     8.9765      0.371     24.188      0.000       8.249       9.704
SALES_SIGNAL_PY                                -0.5734      0.367     -1.564      0.118      -1.292       0.145
ADJUSTED_LIFECYCLE_PY                          -0.0285      0.007     -4.204      0.000      -0.042      -0.015
PCT_OF_LIFECYCLE_REMAINING                      0.4831      0.008     63.031      0.000       0.468       0.498
LIFECYCLE_PRE_PEAK_POST                        -0.0716      0.006    -12.915      0.000      -0.082      -0.061
ADJ_AVG_CLUSTER_LOST_SALES_PY               -4.828e+06   1.55e+07     -0.311      0.756   -3.53e+07    2.56e+07
ADJ_AVG_CLUSTER_TOTAL_SALES_PY              -1.202e+08   1.26e+10     -0.010      0.992   -2.48e+10    2.46e+10
SOLD_SINCE_MAXI                                 0.5255      0.006     81.736      0.000       0.513       0.538
py_gross_sales                                 -0.1151      0.020     -5.668      0.000      -0.155      -0.075
PROJECTED_GROWTH_PCT_PY                         0.8710      0.007    130.291      0.000       0.858       0.884
PROJECTED_GROWTH_PCT_CY                         0.4481      0.006     76

#### Removing all the insignificant variables until only significant variables are left

In [24]:
count=0

while (results.pvalues[results.pvalues > 0.05].index.tolist()):
    print("Iteration: ", count)
    lst = list(results.pvalues[results.pvalues > 0.05].index)
    x.drop(lst, axis='columns', inplace=True)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
    y_train = y_train.tolist()
    y_test = y_test.tolist()
    olsmodel = sm.OLS(y_train,x_train)
    results = olsmodel.fit()
    
    count += 1

Iteration:  0
Iteration:  1


In [25]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.911
Model:                            OLS   Adj. R-squared (uncentered):              0.911
Method:                 Least Squares   F-statistic:                          2.463e+05
Date:                Thu, 31 Mar 2022   Prob (F-statistic):                        0.00
Time:                        14:40:08   Log-Likelihood:                     -4.0745e+06
No. Observations:             1436348   AIC:                                  8.149e+06
Df Residuals:                 1436288   BIC:                                  8.150e+06
Df Model:                          60                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
APPLICATION_COUNT                              0.0344      0.005      7.607      0.000       0.026       0.043
AGE                                            0.0597      0.004     13.353      0.000       0.051       0.068
PCT_COLLEGE                                    0.0305      0.006      5.238      0.000       0.019       0.042
PCT_BLUE_COLLAR                                0.0870      0.006     13.659      0.000       0.075       0.100
SKU_EXISTENCE_PY                             -15.6356      0.046   -343.614      0.000     -15.725     -15.546
SKU_STORE_PDQ_PY                              17.3406      0.047    366.436      0.000      17.248      17.433
AVG_CLUSTER_UNIT_SALES_PY                      1.3387      0.010    140.491      0.000       1.320       1.357
VIO_COMPARED_TO_CLUSTER_PY                     0.0823      0.003     23.794      0.000       0.076       0.089
QTY_SOLD_PPY                                 -12.1348      0.059   -206.201      0.000     -12.250     -12.019
QTY_SOLD_PY                                    8.3553      0.056    149.463      0.000       8.246       8.465
ADJUSTED_LIFECYCLE_PY                         -0.0224      0.007     -3.354      0.001      -0.035      -0.009
PCT_OF_LIFECYCLE_REMAINING                     0.4861      0.007     65.933      0.000       0.472       0.501
LIFECYCLE_PRE_PEAK_POST                       -0.0718      0.006    -12.998      0.000      -0.083      -0.061
SOLD_SINCE_MAXI                                0.5297      0.006     82.437      0.000       0.517       0.542
PROJECTED_GROWTH_PCT_PY                        0.8492      0.007    127.480      0.000       0.836       0.862
PROJECTED_GROWTH_PCT_CY                        0.4414      0.006     75.828      0.000       0.430       0.453
OTHER_UNIT_PLS_LOST_SALES_PPY                  0.7833      0.014     54.310      0.000       0.755       0.812
OTHER_UNIT_PLS_LOST_SALES_PY                   0.1675      0.012     13.791      0.000       0.144       0.191
QTY_SOLD_PY&APPLICATION_COUNT                  1.7408      0.012    148.290      0.000       1.718       1.764
QTY_SOLD_PY&PCT_WHITE                          0.3145      0.013     23.614      0.000       0.288       0.341
QTY_SOLD_PY&AGE                                1.1087      0.012     90.203      0.000       1.085       1.133
QTY_SOLD_PY&PCT_COLLEGE                       -0.7146      0.015    -46.599      0.000      -0.745      -0.685
QTY_SOLD_PY&PCT_BLUE_COLLAR                    0.5207      0.018     29.129      0.000       0.486       0.556
QTY_SOLD_PY&MEDIAN_HOUSEHOLD_INCOME           -1.2782      0.013    -98.591      0.000      -1.304

#### Evaluating OLS model on the test set

In [26]:
y_preds = results.predict(x_test)
residuals = y_test - y_preds

MSE = np.sum(residuals**2)/len(y_test)
print("Mean Squared Error: ", MSE)

MAE = meanabs(y_test, y_preds)
print("Mean Absolute Error: ", MAE)

Mean Squared Error:  21.888566127279194
Mean Absolute Error:  1.1811917350879597


#### Elastic Net Regression

In [27]:
%%time

en = ElasticNet()

model = en.fit(x_train,y_train)
y_pred = en.predict(x_test)

print("R-squared: ", r2_score(y_test,y_pred))
print("Mean Squared Error: ", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error: ", meanabs(y_test, y_pred))

# #print(stime - time.time())

R-squared:  0.8268674073042839
Mean Squared Error:  29.492486871098414
Mean Absolute Error:  1.5832536652354685
CPU times: user 15min 23s, sys: 11.6 s, total: 15min 35s
Wall time: 15.1 s


#### Lasso Regression

In [28]:
%%time

lasso = Lasso()

model = lasso.fit(x_train,y_train)
y_pred = lasso.predict(x_test)

print("R-squared: ", r2_score(y_test,y_pred))
print("Mean Squared Error: ", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error: ", meanabs(y_test, y_pred))

R-squared:  0.8390784015983335
Mean Squared Error:  27.41238985820811
Mean Absolute Error:  1.6362223806386658
CPU times: user 11min 39s, sys: 9.45 s, total: 11min 48s
Wall time: 11.6 s


#### Ridge Regression

In [29]:
%%time

ridge = Ridge()

model = ridge.fit(x_train,y_train)
y_pred = ridge.predict(x_test)

print("R-squared: ", r2_score(y_test,y_pred))
print("Mean Squared Error: ", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error: ", meanabs(y_test, y_pred))

R-squared:  0.8715293167391678
Mean Squared Error:  21.884498351215928
Mean Absolute Error:  1.1810080083635033
CPU times: user 12.3 s, sys: 332 ms, total: 12.7 s
Wall time: 824 ms
